<a href="https://colab.research.google.com/github/Amimo-cell/datascience/blob/main/Actinomycetes_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import pandas as pd
import io
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import f_oneway
from google.colab import files

In [114]:
data = files.upload()

Saving Actinomycetes.final.csv to Actinomycetes.final (2).csv


In [115]:
#converting the file into a dataframe
data= pd.read_csv(io.StringIO(data['Actinomycetes.final (2).csv'].decode('utf-8')))
#view the dataframe
data.head()

,Antibiotics,E.coli,B.subtilis,S. aureus
0,T1,6,11,6
1,T1y,6,13,19
2,T2b,12,13,15
3,T5b,18,20,18
4,T5a,12,6,6


In [116]:
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns
import matplotlib.pyplot as plt

In [117]:
# Clean column names
data.columns = data.columns.str.strip()

# Reshape data to long format
data_melted = data.melt(id_vars=["Antibiotics"], var_name="Isolate", value_name="Zone_Diameter")

data.head()

,Antibiotics,E.coli,B.subtilis,S. aureus
0,T1,6,11,6
1,T1y,6,13,19
2,T2b,12,13,15
3,T5b,18,20,18
4,T5a,12,6,6


In [118]:
# Clean and check the data
print("Initial data shape:", data_melted.shape)

Initial data shape: (51, 3)


In [119]:
print(data_melted.isnull().sum())

Antibiotics      0
Isolate          0
Zone_Diameter    0
dtype: int64


In [120]:
group_stats = data_melted.groupby(['Antibiotics', 'Isolate'])['Zone_Diameter'].agg(['count', 'std'])
print(group_stats)

                            count  std
Antibiotics     Isolate               
Amikyacin       B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
Ampicillin      B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
CAZ             B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
COT             B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
Cefuroxime      B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
Cephtriaxone    B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
Chloromphenicol B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus       1  NaN
Gentamycin      B.subtilis      1  NaN
                E.coli          1  NaN
                S. aureus

In [121]:
# Convert to numeric to avoid type issues
data_melted['Zone_Diameter'] = pd.to_numeric(data_melted['Zone_Diameter'], errors='coerce')



In [126]:
# Final data checks
print("Data shape after cleaning:", data_melted.shape)
print("Any NaNs left?:", data_melted.isnull().sum().sum())
print("Unique antibiotics:", data_melted['Antibiotics'].unique())
print("Unique isolates:", data_melted['Isolate'].unique())


Data shape after cleaning: (51, 3)
Any NaNs left?: 0
Unique antibiotics: ['T1' 'T1y' 'T2b' 'T5b' 'T5a' 'Th4' 'T4b' 'sterile water ' 'Ampicillin '
 'Chloromphenicol' 'COT ' 'Penicillin ' 'Gentamycin ' 'CAZ' 'Amikyacin'
 'Cephtriaxone' 'Cefuroxime']
Unique isolates: ['E.coli' 'B.subtilis' 'S. aureus']


In [123]:
#Null hypotheses (H₀):
#There is no significant difference in inhibition zones across antibiotics.
#There is no significant difference in inhibition zones across microorganisms.
#There is no interaction effect between antibiotic type and microorganism type on inhibition zones.
#Alternative hypotheses (H₁):
#There is a significant difference in inhibition zones across antibiotics.
#There is a significant difference in inhibition zones across microorganisms.
#There is a significant interaction effect between antibiotic type and microorganism type.

In [125]:
# Check for groups with all zeros
zero_groups = data_melted.groupby(['Antibiotics', 'Isolate']).filter(lambda x: (x['Zone_Diameter'] == 0).all())

# Print out the zero-value groups
print("Groups with all zero values:")
print(zero_groups)

Groups with all zero values:
Empty DataFrame
Columns: [Antibiotics, Isolate, Zone_Diameter]
Index: []


In [124]:
model = ols('Zone_Diameter ~ C(Antibiotics) * C(Isolate)', data=data_melted).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
 # Print ANOVA results
print("Two-Way ANOVA Results:")
print(anova_table)


/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


ValueError: array must not contain infs or NaNs